In [ ]:
USER = 'Paolo'
LOSS = 'mean_squared_error'

In [ ]:
import sys
import os
if colab := 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    # BASE_PATH = '/content/drive/Shareddrives/Project_ML_23/' + USER + '/machine-learning-project'
    BASE_PATH = '/content/drive/My Drive/machine-learning-project'
    sys.path.insert(0, BASE_PATH)
    N_JOBS = -1
    sys.path.insert(0, BASE_PATH + '/src/utils')
    !pip install optuna
    !pip install scikit-learn
    !pip install scikeras
    TRAIN_DATA = os.path.join(BASE_PATH, 'datasets', 'cup2023','ML-CUP23-TR.csv')
    IMAGES_FOLDER = os.path.join(BASE_PATH, 'images', 'cup2023', 'neural_network')
    MODEL_FOLDER = os.path.join(BASE_PATH, 'trained_models', 'cup2023')
else :
    N_JOBS = -1
    TRAIN_DATA = os.path.join('..', '..', 'datasets', 'cup2023', 'ML-CUP23-TR.csv')
    IMAGES_FOLDER = os.path.join('..', '..', 'images', 'cup2023', 'neural_network')
    MODEL_FOLDER = os.path.join('..', '..', 'trained_models', 'cup2023')

In [ ]:
if colab := 'google.colab' in sys.modules:
    sys.path.append(BASE_PATH + '/src/utils')
else:
    sys.path.append('../utils')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme(style='darkgrid')
from keras import backend as K


from utils import save_plot, mean_euclidean_error, root_mean_squared_error, multidim_r2, set_random_state, mean_squared_error,save_optuna_study
from NN import NeuralNetwork
from joblib import dump

set_random_state(42)

In [ ]:
# To skip the first column (row indexes)
columns_to_read = list(range(1, 14))

df_train = pd.read_csv(TRAIN_DATA, header=None, comment='#', usecols=columns_to_read, delimiter=',')
df_train = df_train.astype('float64') # casting
df_train.head()

In [ ]:
features = ['feature_' + str(i) for i in range(1, 11)]
targets = ['target_x', 'target_y', 'target_z']

# Rename columns
new_column_names = features + targets
df_train.columns = new_column_names

df_train.head()

In [ ]:
from sklearn.model_selection import train_test_split

X = df_train[features].to_numpy()
y = df_train[targets].to_numpy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create model

In [ ]:
from sklearn.preprocessing import RobustScaler
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('scaler', RobustScaler()),
    ('neuralnetwork', NeuralNetwork(loss=LOSS, verbose=0))
])

In [ ]:
param_grid = {
    'neuralnetwork__architecture': [
        (128, 64),
        (128, 128),
        (256, 128)
    ],
    'neuralnetwork__activation': ['relu'],
    'neuralnetwork__learning_rate': [1e-3, 1e-2],
    'neuralnetwork__momentum': [0.5,  1.0],
    'neuralnetwork__use_nesterov': [True, False],
    'neuralnetwork__weight_decay': [1e-3, 1e-2],
    'neuralnetwork__input_dimension': [10],
    'neuralnetwork__output_dimension': [3],
    'neuralnetwork__dropout_input_rate': [0.0, 0.1],
    'neuralnetwork__dropout_hidden_rate': [
        (0.1, 0.1),
        (0.2, 0.2)
    ],
    'neuralnetwork__patience': [5],
    'neuralnetwork__epochs': [50, 100, 150],
    'neuralnetwork__batch_size': [16, 32],
    'neuralnetwork__early' : [False]
}

In [ ]:
from sklearn.metrics import make_scorer

mee_scorer = make_scorer(mean_euclidean_error, greater_is_better=False)
rmse_scorer = make_scorer(root_mean_squared_error, greater_is_better=False)
multidim_r2_scorer = make_scorer(multidim_r2, greater_is_better=True)
mse_scorer = make_scorer(mean_squared_error, greater_is_better=False)

scoring = {'MEE': mee_scorer, 'R2': multidim_r2_scorer, 'RMSE': rmse_scorer, 'MSE': mse_scorer}

In [ ]:
from sklearn.model_selection import GridSearchCV, KFold

cv = KFold(n_splits=5, shuffle=True, random_state=42)

grid_search = GridSearchCV(
    pipeline,
    param_grid=param_grid,
    cv=cv,
    scoring=scoring,
    refit='MEE',
    n_jobs=N_JOBS,
    verbose=0
)

grid_search.fit(X_train, y_train)

In [ ]:
results = grid_search.cv_results_

results_df = pd.DataFrame(results)
csv_filename_result =os.path.join(MODEL_FOLDER, 'NN_Cup_Grid_Results.csv')
csv_filename_loss = os.path.join(MODEL_FOLDER, 'NN_Cup_Grid_Loss.csv')
results_df.to_csv(csv_filename_result, index=False)

In [ ]:
best_params = grid_search.best_params_
best_index = grid_search.best_index_

mee = grid_search.best_score_
r2 = grid_search.cv_results_['mean_test_R2'][best_index]
rmse = grid_search.cv_results_['mean_test_RMSE'][best_index]
mse = grid_search.cv_results_['mean_test_MSE'][best_index]

print('Best parameters:', best_params)
print('Best MEE score:', abs(mee))
print('R2:', abs(r2))
print('RMSE:', abs(rmse))
print('MSE:', abs(mse))

# Learning Curve

In [ ]:
train_scores = []

best_params = grid_search.best_params_

params = {
    'architecture': best_params['neuralnetwork__architecture'],
    'activation': best_params['neuralnetwork__activation'],
    'learning_rate': best_params['neuralnetwork__learning_rate'],
    'momentum': best_params['neuralnetwork__momentum'],
    'use_nesterov': best_params['neuralnetwork__use_nesterov'],
    'weight_decay': best_params['neuralnetwork__weight_decay'],
    'input_dimension': best_params['neuralnetwork__input_dimension'],
    'output_dimension': best_params['neuralnetwork__output_dimension'],
    'dropout_input_rate': best_params['neuralnetwork__dropout_input_rate'],
    'dropout_hidden_rate': best_params['neuralnetwork__dropout_hidden_rate'],
    'patience': best_params['neuralnetwork__patience'],
    'epochs': 800,
    'batch_size': best_params['neuralnetwork__batch_size'],
    'early': True
}

In [ ]:
# Cross-validation with the best parameters
i = 0
final_model = []

for train_index, test_index in cv.split(X_train):
    X_train_1, X_test_1 = X_train[train_index], X_train[test_index]
    y_train_1, y_test_1 = y_train[train_index], y_train[test_index]
    i = i + 1

    # Generate 5 possible splits and foreach save a model and history
    pipeline_ = Pipeline([
        ('scaler', RobustScaler()),
        ('neuralnetwork', NeuralNetwork(verbose=0, **params, loss=LOSS, validation_data=(X_test_1, y_test_1)))
    ])

    # Save different model for each fold to make ensemble model later
    final_model.append(pipeline_.fit(X_train_1, y_train_1))
    train_scores.append(pipeline_.named_steps['neuralnetwork'].history.history)

In [ ]:
train_scores_df = pd.DataFrame(train_scores)
train_scores_df.to_csv(csv_filename_loss, index=False)

In [ ]:
loss, val_loss, mee, val_mee = [], [], [], []

for i in range(len(train_scores_df)):
    loss.append(train_scores_df.iloc[i]['loss'])
    val_loss.append(train_scores_df.iloc[i]['val_loss'])
    mee.append(train_scores_df.iloc[i]['mean_euclidean_error'])
    val_mee.append(train_scores_df.iloc[i]['val_mean_euclidean_error'])

df_loss = pd.DataFrame(loss).T
df_val_loss = pd.DataFrame(val_loss).T
df_mee = pd.DataFrame(mee).T
df_val_mee = pd.DataFrame(val_mee).T

mean_loss = df_loss.mean(axis=1)
stdev_loss = df_loss.std(axis=1)
mean_val_loss = df_val_loss.mean(axis=1)
stdev_val_loss = df_val_loss.std(axis=1)

mean_mee = df_mee.mean(axis=1)
stdev_mee = df_mee.std(axis=1)
mean_val_mee = df_val_mee.mean(axis=1)
stdev_val_mee = df_val_mee.std(axis=1)

plt.figure(figsize=(12, 5))

# loss and val_loss
plt.subplot(1, 2, 1)
plt.plot(mean_loss, label='Loss(MSE)', linestyle='--')
plt.fill_between(range(len(mean_loss)), mean_loss-stdev_loss, mean_loss+stdev_loss, alpha=0.2)
plt.plot(mean_val_loss, label='Val Loss', linestyle='-.')
plt.fill_between(range(len(mean_val_loss)), mean_val_loss-stdev_val_loss, mean_val_loss+stdev_val_loss, alpha=0.2)
plt.title('Learning Curve for Loss (MSE)')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# mee and val mee
plt.subplot(1, 2, 2)
plt.plot(mean_mee, label='MEE', linestyle='-')
plt.fill_between(range(len(mean_mee)), mean_mee-stdev_mee, mean_mee+stdev_mee, alpha=0.2)
plt.plot(mean_val_mee, label='Val MEE', linestyle='-.')
plt.fill_between(range(len(mean_val_mee)), mean_val_mee-stdev_val_mee, mean_val_mee+stdev_val_mee, alpha=0.2)
plt.title('Learning Curve for MEE')
plt.xlabel('Epochs')
plt.ylabel('MEE')
plt.legend()

plt.tight_layout()
save_plot(plt, IMAGES_FOLDER, 'learning_curve_nn_cup_grid')
plt.show()

# Test of the model

In [ ]:
y_pred_ensemble = np.zeros_like(y_test)

for model in final_model:
    y_pred = model.predict(X_test)
    y_pred_ensemble += y_pred

y_pred_ensemble /= len(final_model)

mse_ensemble = mean_squared_error(y_test, y_pred_ensemble)
r2_ensemble = multidim_r2(y_test, y_pred_ensemble)
rmse_ensemble = root_mean_squared_error(y_test, y_pred_ensemble)
mee_ensemble = mean_euclidean_error(y_test, y_pred_ensemble)

print('Ensemble MSE:', mse_ensemble)
print('Ensemble R2:', r2_ensemble)
print('Ensemble RMSE:', rmse_ensemble)
print('Ensemble MEE:', mee_ensemble)

In [ ]:
results = {
    'Metric': ['Ensemble MEE', 'Ensemble R2', 'Ensemble RMSE', 'Ensemble MSE'],
    'Value': [mee_ensemble, r2_ensemble, rmse_ensemble, mse_ensemble]
}

results_df = pd.DataFrame(results)

# Save the dataframe as an Excel file
excel_filename = os.path.join(MODEL_FOLDER, 'results_test_grid_nn.xlsx')
results_df.to_excel(excel_filename, index=False)

## Ensamble of the final model using the whole dataset

In [ ]:
# Cross-validation with the best parameters
i = 0
final_model_ = []
test_scores = []
for train_index, test_index in cv.split(X):
    X_train_1, X_test_1 = X[train_index], X[test_index]
    y_train_1, y_test_1 = y[train_index], y[test_index]
    i = i + 1

    # Generate 5 possible splits and foreach save a model and history
    pipeline_test = Pipeline([
        ('scaler', RobustScaler()),
        ('neuralnetwork', NeuralNetwork(verbose=0, **params, loss=LOSS, validation_data=(X_test_1, y_test_1)))
    ])

    # Save different model for each fold to make ensemble model later
    final_model_.append(pipeline_test.fit(X_train_1, y_train_1))
    test_scores.append(pipeline_test.named_steps['neuralnetwork'].history.history)

test_scores_df = pd.DataFrame(test_scores)

# Save model

In [ ]:
# Save the model best parameters
best_params = grid_search.best_params_
best_params_df = pd.DataFrame([best_params])

csv_filename = os.path.join(MODEL_FOLDER, 'best_params_Grid_NN.csv')
best_params_df.to_csv(csv_filename, index=False)

In [ ]:
model_path = os.path.join(MODEL_FOLDER, 'NN_grid_model.joblib')
dump(final_model, model_path, compress=3)

model_path = os.path.join(MODEL_FOLDER, 'NN_grid_model_final.joblib')
dump(final_model, model_path, compress=3)